<a href="https://colab.research.google.com/github/leonardolunamoreno/Computadoras-y-programaci-n2021-grupo-1157/blob/main/GUI_Afinador_de_guitarra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GUI de Afinador de guitarra, versión 1

En está versión del afinador de guitarra se muestra una GUI sencila para observar las pulsaciiones en tiempo real, no obstante, en esta versión del afinador es necesario presionar continuamente el botón " Escuchar " antes de afinar una cuerda determinada, también será necesario tener un micrófono conectado a la computadora, o tener los permisos habilitados en la computadora para utilizar el mismo.

In [ ]:
import numpy as np
import sounddevice as sd
import math
import tkinter as tk
from tkinter import ttk

# Afinaciones predefinidas
TUNINGS = {
    "E Standard": {6: 82.41, 5: 110.00, 4: 146.83, 3: 196.00, 2: 246.94, 1: 329.63},
    "Drop D":     {6: 73.42, 5: 110.00, 4: 146.83, 3: 196.00, 2: 246.94, 1: 329.63},
    "Eb Standard":{6: 77.78, 5: 103.83, 4: 138.59, 3: 185.00, 2: 233.08, 1: 311.13},
    "Drop C# (Over Now)": {6: 69.30, 5: 103.83, 4: 138.59, 3: 185.00, 2: 233.08, 1: 311.13}
}

# Procesamiento FFT
def parabolic_interpolation(mags, peak_idx):
    if peak_idx <= 0 or peak_idx >= len(mags)-1:
        return peak_idx
    alpha = mags[peak_idx-1]
    beta  = mags[peak_idx]
    gamma = mags[peak_idx+1]
    p = 0.5 * (alpha - gamma) / (alpha - 2*beta + gamma)
    return peak_idx + p

def detectar_frecuencia(duracion=0.8, fs=44100):
    audio = sd.rec(int(duracion * fs), samplerate=fs, channels=1)
    sd.wait()

    señal = audio.flatten().astype(float)
    señal -= np.mean(señal)

    N = len(señal)
    ventana = np.hanning(N)
    señal *= ventana

    zpad = 4 * N
    fft = np.fft.rfft(señal, n=zpad)
    mags = np.abs(fft)
    freqs = np.fft.rfftfreq(zpad, 1/fs)

    mags[:2] = 0
    idx = np.argmax(mags)
    idx_ref = parabolic_interpolation(mags, idx)

    return freqs[int(idx_ref)]

def comparar(f_detectada, f_objetivo):
    cents = 1200 * math.log2(f_detectada / f_objetivo)
    if abs(cents) < 5:
        estado = "OK"
    elif cents > 0:
        estado = "ALTA"
    else:
        estado = "BAJA"
    return cents, estado

# ---------------- GUI ------------------

class AfinadorGUI:
    def __init__(self, master):
        self.master = master
        master.title("Afinador de Guitarra")

        # Afinación
        self.tuning_var = tk.StringVar(value="E Standard")
        ttk.Label(master, text="Afinación:").pack()
        self.menu = ttk.Combobox(master, textvariable=self.tuning_var, values=list(TUNINGS.keys()))
        self.menu.pack()

        # Cuerda
        self.cuerda_var = tk.IntVar(value=6)
        ttk.Label(master, text="Cuerda a afinar (1–6):").pack()
        self.spin = tk.Spinbox(master, from_=1, to=6, textvariable=self.cuerda_var)
        self.spin.pack()

        # Botón de iniciar medición
        self.btn = tk.Button(master, text="Escuchar", command=self.medir)
        self.btn.pack(pady=10)

        # Display grande
        self.label_freq = tk.Label(master, text="— Hz", font=("Arial", 24))
        self.label_freq.pack()

        self.label_estado = tk.Label(master, text="", font=("Arial", 20))
        self.label_estado.pack(pady=5)

    def medir(self):
        tuning = TUNINGS[self.tuning_var.get()]
        cuerda = self.cuerda_var.get()
        objetivo = tuning[cuerda]

        f = detectar_frecuencia()
        cents, estado = comparar(f, objetivo)

        self.label_freq.config(text=f"{f:.2f} Hz  |  objetivo {objetivo:.2f} Hz")

        if estado == "OK":
            self.label_estado.config(text=f"{cents:+.1f} cents → OK", fg="green")
        elif estado == "ALTA":
            self.label_estado.config(text=f"{cents:+.1f} cents → ALTA", fg="red")
        else:
            self.label_estado.config(text=f"{cents:+.1f} cents → BAJA", fg="blue")


# Lanzar GUI
root = tk.Tk()
app = AfinadorGUI(root)
root.mainloop()


GUI de afinador de guitarra, versión 2

In [ ]:
import numpy as np
import sounddevice as sd
import math
import tkinter as tk
from tkinter import ttk

# Afinaciones predefinidas
TUNINGS = {
    "E Standard": {6: 82.41, 5: 110.00, 4: 146.83, 3: 196.00, 2: 246.94, 1: 329.63},
    "Drop D":     {6: 73.42, 5: 110.00, 4: 146.83, 3: 196.00, 2: 246.94, 1: 329.63},
    "Eb Standard":{6: 77.78, 5: 103.83, 4: 138.59, 3: 185.00, 2: 233.08, 1: 311.13},
    "Drop C# (Over Now)": {6: 69.30, 5: 103.83, 4: 138.59, 3: 185.00, 2: 233.08, 1: 311.13}
}

# Procesamiento FFT
def parabolic_interpolation(mags, peak_idx):
    if peak_idx <= 0 or peak_idx >= len(mags)-1:
        return peak_idx
    alpha = mags[peak_idx-1]
    beta  = mags[peak_idx]
    gamma = mags[peak_idx+1]
    p = 0.5 * (alpha - gamma) / (alpha - 2*beta + gamma)
    return peak_idx + p

def detectar_frecuencia(duracion=0.8, fs=44100):
    audio = sd.rec(int(duracion * fs), samplerate=fs, channels=1)
    sd.wait()

    señal = audio.flatten().astype(float)
    señal -= np.mean(señal)

    N = len(señal)
    ventana = np.hanning(N)
    señal *= ventana

    zpad = 4 * N
    fft = np.fft.rfft(señal, n=zpad)
    mags = np.abs(fft)
    freqs = np.fft.rfftfreq(zpad, 1/fs)

    mags[:2] = 0
    idx = np.argmax(mags)
    idx_ref = parabolic_interpolation(mags, idx)

    return freqs[int(idx_ref)]

def comparar(f_detectada, f_objetivo):
    cents = 1200 * math.log2(f_detectada / f_objetivo)
    if abs(cents) < 5:
        estado = "OK"
    elif cents > 0:
        estado = "ALTA"
    else:
        estado = "BAJA"
    return cents, estado




# ---------------- GUI ------------------

class AfinadorGUI:
    def __init__(self, master):
        self.master = master
        self.escuchando = False
        master.title("Afinador de Guitarra")

        # Afinación
        self.tuning_var = tk.StringVar(value="E Standard")
        ttk.Label(master, text="Afinación:").pack()
        self.menu = ttk.Combobox(master, textvariable=self.tuning_var, values=list(TUNINGS.keys()))
        self.menu.pack()

        # Cuerda
        self.cuerda_var = tk.IntVar(value=6)
        ttk.Label(master, text="Cuerda a afinar (1–6):").pack()
        self.spin = tk.Spinbox(master, from_=1, to=6, textvariable=self.cuerda_var)
        self.spin.pack()

        # Botón de iniciar medición
        self.btn = tk.Button(master, text="Escuchar", command=self.toggle_escuchar)
        self.btn.pack(pady=10)

        # Display grande
        self.label_freq = tk.Label(master, text="— Hz", font=("Arial", 24))
        self.label_freq.pack()

        self.label_estado = tk.Label(master, text="", font=("Arial", 20))
        self.label_estado.pack(pady=5)

    def toggle_escuchar(self):
        self.escuchando = not self.escuchando

        if self.escuchando:
            self.btn.config(text="Detener")
            self.actualizar()
        else:
            self.btn.config(text="Escuchar")


    def actualizar(self):
        if not self.escuchando:
            return

        tuning = TUNINGS[self.tuning_var.get()]
        cuerda = self.cuerda_var.get()
        objetivo = tuning[cuerda]

        f = detectar_frecuencia()
        cents, estado = comparar(f, objetivo)

        self.label_freq.config(
            text=f"{f:.2f} Hz  |  objetivo {objetivo:.2f} Hz"
        )

        if estado == "OK":
            self.label_estado.config(text=f"{cents:+.1f} cents → OK", fg="green")
        elif estado == "ALTA":
            self.label_estado.config(text=f"{cents:+.1f} cents → ALTA", fg="red")
        else:
            self.label_estado.config(text=f"{cents:+.1f} cents → BAJA", fg="blue")

    # 👇 vuelve a llamarse sola cada 300 ms
        self.master.after(300, self.actualizar)

# Lanzar GUI
root = tk.Tk()
app = AfinadorGUI(root)
root.mainloop()
